# Capstone project week 1

In [43]:
import pandas as pd
import numpy as np

print("Hello Capstone Project Course!")

Hello Capstone Project Course!


In [44]:
# returns a list of dataframes
data_orig = pd.read_html("http://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")#, attrs = {'id': 'table'})

# get the table
data_orig = data_orig[0]
data_orig.head(20)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
8,M8A,Not assigned,Not assigned
9,M9A,Etobicoke,Islington Avenue


In [45]:
# make a copy and work on this from now on
data = data_orig.copy(deep=True)

In [46]:
data.shape

(287, 3)

In [47]:
# The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
data.columns = ['PostalCode', 'Borough', 'Neighborhood']

In [48]:
# Ignore cells with a borough that is "Not assigned"
data = data[data['Borough']!='Not assigned']

In [49]:
data.head(20)

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
9,M9A,Etobicoke,Islington Avenue
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
13,M3B,North York,Don Mills North


In [50]:
# More than one neighborhood can exist in one postal code area. These neighborhoods should be combined into one row and separated with a comma.

def aggregate_neighbourhoods(df):
    # create a dictionary to hold the postal codes as keys and (Boroughs, Neighborhoods) as values
    D = dict.fromkeys(data['PostalCode'].unique())
    # loop through the rows of the dataframe df
    for item in df:
        # get the list of Neighborhoods and join the list items with ", "
        hds = ", ".join(list(item[1]['Neighborhood'].unique()))
        # now get the list of boroughs and join the list items with ", "
        bor = ", ".join(list(item[1]['Borough'].unique()))
        # populate the dictionary
        D[item[0]] = (bor, hds)
        
    new_df = pd.DataFrame.from_dict(D, orient='index')  # without orient='index' the dict keys will be the columns of new_df
    new_df.reset_index(inplace=True)  # reset the index to row numbers instead of postal codes
    new_df.columns = ['Postal Code', 'Borough', 'Neighborhood']  # name the columns
    return new_df


df = data.groupby(['PostalCode'])  # group the data by the postal codes
data = aggregate_neighbourhoods(df)  # pass the dataframe in the function to get the final dataframe
data.head(20)

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


In [51]:
data.shape

(103, 3)

In [52]:
geo_coords = pd.read_csv('C:/Users/maidu/IBM_Project/Geospatial_Coordinates.csv')

# sanity check. The below should not return an error if the dataframe data is correct
assert(geo_coords.shape[0] == data.shape[0])

geo_coords.head(12)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [53]:
analysis_data = data.merge(geo_coords, on='Postal Code')
analysis_data.head(12)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
